Khai báo các thư viện cần thiết

In [1]:
import os
import numpy as np

Khai báo đường dẫn tệp dữ liệu

In [2]:
data_path = 'D:/Code/python/mat3533/project/data/'

train_images_path = os.path.join(data_path, 'train-images-idx3-ubyte.gz')
train_labels_path = os.path.join(data_path, 'train-labels-idx1-ubyte.gz')

test_images_path = os.path.join(data_path, 't10k-images-idx3-ubyte.gz')
test_labels_path = os.path.join(data_path, 't10k-labels-idx1-ubyte.gz')

Hàm lấy dữ liệu

In [3]:
def get_mnist_data(images_path, labels_path, num_images, shuffle=False, _is=True, image_size=28):
    """
    This shuffle param is active when .gz is downloaded at:
    - 'http://yann.lecun.com/exdb/mnist/'
    - This function return random num_images in 60000 or 10000
    """
    # read data
    import gzip # to decompress gz (zip) file
    # open file training to read training data
    f_images = gzip.open(images_path,'r')
    # skip 16 first bytes because these are not data, only header infor
    f_images.read(16)
    # general: read num_images data samples if this parameter is set;
    # if not, read all (60000 training or 10000 test)
    real_num = num_images if not shuffle else (60000 if _is else 10000)
    # read all data to buf_images (28x28xreal_num)
    buf_images = f_images.read(image_size * image_size * real_num)
    # images
    images = np.frombuffer(buf_images, dtype=np.uint8).astype(np.float32)
    images = images.reshape(real_num, image_size, image_size,)
    # Read labels
    f_labels = gzip.open(labels_path,'r')
    f_labels.read(8)
    labels = np.zeros((real_num)).astype(np.int64)
    # rearrange to correspond the images and labels
    for i in range(0, real_num):
        buf_labels = f_labels.read(1)
        labels[i] = np.frombuffer(buf_labels, dtype=np.uint8).astype(np.int64)

    # shuffle to get random images data
    if shuffle is True:
        rand_id = np.random.randint(real_num, size=num_images)
        images = images[rand_id, :]
        labels = labels[rand_id,]
    # change images data to type of vector 28x28 dimentional
    images = images.reshape(num_images, image_size * image_size)
    return images, labels

Lấy dữ liệu train, test từ tệp dữ liệu

In [4]:
train_images, train_labels = get_mnist_data(
    train_images_path, train_labels_path, 60000)
test_images, test_labels = get_mnist_data(
    test_images_path, test_labels_path, 10000, _is=False)
print(train_images.shape, train_labels.shape)
print(test_images.shape, test_labels.shape)

(60000, 784) (60000,)
(10000, 784) (10000,)


Chuẩn hóa dữ liệu

In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
train_x = scaler.fit_transform(train_images)
test_x = scaler.fit_transform(test_images)

print(train_x.shape)
print(test_x.shape)

(60000, 784)
(10000, 784)


Giảm số chiều dữ liệu về 100 chiều

In [6]:
from sklearn.decomposition import PCA

pca = PCA(n_components=100)
pca.fit(train_images)
X_train = pca.transform(train_images)
X_test = pca.transform(test_images)

print(X_train.shape)
print(X_test.shape)

(60000, 100)
(10000, 100)


Khởi tạo mô hình ANN

In [7]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten

model = Sequential()

model.add(Flatten(input_shape = (100,)))
model.add(Dense(128,activation = 'relu'))
model.add(Dense(64,activation = 'relu'))
model.add(Dense(32,activation = 'relu'))
model.add(Dense(10,activation = 'softmax')) 

c:\Users\Admin\anaconda3\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Tổng quan mô hình

In [8]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        12,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │           330 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,594 (92.16 KB)

 Trainable params: 23,594 (92.16 KB)

 Non-trainable params: 0 (0.00 B)

In [10]:
model.compile(loss = 'sparse_categorical_crossentropy', 
              optimizer = 'Adam', 
              metrics = ['accuracy'])

Huấn luyện mô hình bằng tập dữ liệu train

In [11]:
model.fit(X_train, train_labels, epochs= 10 , validation_split = .2)

Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.6403 - loss: 10.5137 - val_accuracy: 0.8250 - val_loss: 0.6168
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8594 - loss: 0.5106 - val_accuracy: 0.8952 - val_loss: 0.4194
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9092 - loss: 0.3099 - val_accuracy: 0.9233 - val_loss: 0.3166
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9283 - loss: 0.2451 - val_accuracy: 0.9319 - val_loss: 0.2687
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9451 - loss: 0.1947 - val_accuracy: 0.9438 - val_loss: 0.2435
Epoch 6/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9546 - loss: 0.1619 - val_accuracy: 0.9503 - val_loss: 0.1832
Epoch 7/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9651 - loss: 0.1211 - val_accuracy: 0.9527 - val_loss: 0.1883
Epoch 8/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9709 - loss: 0.1001 -

Sử dụng mô hình đã huấn luyện dự đoán tập dữ liệu test

In [12]:
y_pred = model.predict(X_test)
y_pred = y_pred.argmax(axis = 1)
print(y_pred)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
[7 2 1 ... 4 5 6]


Tính độ chính xác của mô hình

In [13]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score

accuracy = accuracy_score(test_labels, y_pred)
print("Accuracy:", accuracy)

confusion_matrix = confusion_matrix(test_labels, y_pred)
print("Confusion Matrix:")
print(confusion_matrix)

precision = precision_score(test_labels, y_pred, average='macro')
print("Precision:", precision)

recall = recall_score(test_labels, y_pred, average='macro')
print("Recall:", recall)

Accuracy: 0.9637
Confusion Matrix:
[[ 957    0    9    0    2    2    3    4    3    0]
 [   0 1123    2    2    0    0    1    1    5    1]
 [   0    6  993   13    4    0    3    6    7    0]
 [   0    1    6  977    0    9    1    4    5    7]
 [   1    1    0    0  959    0    6    3    1   11]
 [   4    3    0   13    0  840    7    2   16    7]
 [   4    2    1    1    4    5  938    0    3    0]
 [   0    9    8   12   10    0    1  977    2    9]
 [   4    3   11    5    6    7    4    2  930    2]
 [   1    6    1   10   21    4    2   13    8  943]]
Precision: 0.9637442785527444
Recall: 0.9632702849331995
